In [ ]:
!pip install -q google-genai google-adk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00


In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)

In [ ]:
# Cria um serviço de sessão em memória

session_service = InMemorySessionService()

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

async def call_agent(agent: Agent, message_text: str) -> str:
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Lista todos os modelos disponíveis atualmente

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation


In [ ]:
MODELO_RAPIDO = "gemini-2.0-flash"
MODELO_ROBUSTO = "gemini-2.5-pro-preview-03-25"

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

async def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model=MODELO_RAPIDO,
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias e informações sobre o IP ou domínio informado.
        Obtenha pelo menos 5 informações atuais. Considere atual informações publicadas até um mês antes da data de hoje.
        Se um determinado IP ou Domínio tiver poucas informações publicadas, busque informações relacionadas mais antigas.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos = await call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Avaliador de informações --- #
################################################

async def agente_investigador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_investigador",
        model=MODELO_ROBUSTO,
        instruction="""
        Você é um analista de segurança da informação, especialista em inteligência de ameaças.
        Você recebe uma lista de informações sobre um domínio ou IP do agente buscador e,
        - Para cada uma das informações, você deve usar a ferramenta
        de busca do Google (google_search) para buscar os pontos mais relevantes
        que poderíamos abordar em uma análise de reputação sobre este IP ou domínio.
        Você também pode usar o (google_search) para encontrar mais
        informações sobre o IP ou domínio e aprofundar.
        - Depois de terminar a busca, você irá escolher as informações que sejam mais relevantes, evitando duplicações, e atuais e irá retornar
        as informações, seus pontos mais relevantes, para ajudar a avaliar a reputação malicioso ou não deste IP ou domínio.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_investigador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = await call_agent(planejador, entrada_do_agente_investigador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

async def agente_avaliador(topico, plano_de_post):
    redator = Agent(
        name="agente_avaliador",
        model=MODELO_ROBUSTO,
        instruction="""
            Você é um Analista de Resposta a incidentes especializado em tratar IPs e Domínios malicisos.
            Utilize as informações o ip/domínio fornecidos e com base nisso,
            avalie se o IP ou domínio pode ser considerado malicioso ou não. Se necessário acesse sites de reputação como ao AbuseIPDB para validar e comparar seu veredito.
            Crie um resumo breve informando se o IP ou Domínio foi considerado malicioso ou não e justifique.
            """,
        description="Agente avaliador de reputação"
    )
    entrada_do_agente_avaliador = f"Tópico: {topico}\nPlano de post: {plano_de_post}"

    # Executa o agente
    post_final = await call_agent(redator, entrada_do_agente_avaliador)
    return post_final

In [ ]:
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

In [ ]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀\n")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o IP ou Domínio sobre o qual você quer avaliar a reputação: ")

# Inserir lógica do sistema de agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nVamos avaliar o seguinte IP/Domínio: {topico}")

    lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = await agente_investigador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    post_redigido = await agente_avaliador(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (avaliador) ---\n")
    display(to_markdown(post_redigido))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 3 Agentes 🚀

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: 179.248.212.19

Maravilha! Vamos avaliar o seguinte IP/Domínio: 179.248.212.19

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer informações relevantes sobre o IP 179.248.212.19, farei algumas pesquisas no Google para identificar informações sobre sua localização, reputação, possíveis usos e outras informações relevantes.
> 
> 
> Com base nas informações encontradas, aqui estão algumas informações sobre o IP 179.248.212.19, com foco nos dados mais recentes:
> 
> 1.  **Reputação:** O IP 179.248.212.19 está listado em algumas blacklists, incluindo a DataPlane TELNET login blacklist e a Spamhaus PBL blacklist (dados de 2025-05-31). Isso sugere que o IP pode ter sido associado a atividades como tentativas de login TELNET não solicitadas e envio de e-mails não autenticados.
> 2.  **Geolocation:** Informações de geolocalização indicam que este IP está associado ao Brasil.
> 3.  **Provedor de serviços:** O provedor de serviços de internet (ISP) associado a este IP é a Tim Celular S.A.
> 4.  **Informações adicionais:** O IP está incluído no intervalo de endereços IP 179.248.0.0 - 179.248.255.255.
> 5.  **IP reputation checker:** A reputação do IP pode ser verificada em ferramentas como CloudFilt Bot Management.
> 
> É importante notar que as informações sobre IP podem mudar com o tempo.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nas informações fornecidas sobre o IP 179.248.212.19, realizei buscas para aprofundar a análise de reputação.
> 
> **Análise das Informações Fornecidas e Buscas Adicionais:**
> 
> 1.  **Reputação (Blacklists):**
>     *   **DataPlane TELNET login blacklist:** Estar nesta lista indica que o IP foi observado tentando realizar logins via TELNET, um protocolo frequentemente explorado por ser inseguro. Isso é um forte indicador de atividade suspeita ou maliciosa, como tentativas de acesso não autorizado a dispositivos.
>     *   **Spamhaus PBL (Policy Block List):** A PBL lista blocos de IPs que não deveriam enviar e-mails diretamente para servidores de terceiros (por exemplo, IPs de usuários finais dinâmicos). Embora não signifique necessariamente que o IP seja malicioso por si só, e-mails originados dele podem ser bloqueados ou marcados como spam. É comum que IPs residenciais dinâmicos estejam nesta lista para prevenir o envio de spam por máquinas infectadas.
> 
> 2.  **Geolocalização (Brasil) e Provedor de Serviços (Tim Celular S.A.):**
>     *   O IP 179.248.212.19 está geolocalizado no Brasil e é fornecido pela Tim Celular S.A. Trata-se de um endereço IP provavelmente dinâmico, comumente atribuído a dispositivos móveis ou usuários residenciais. IPs dinâmicos podem mudar de usuário com frequência, o que significa que a reputação pode não estar ligada a um único indivíduo ou dispositivo por um longo período. No entanto, se um dispositivo conectado através deste IP estiver comprometido, ele pode ser usado para atividades maliciosas.
> 
> 3.  **Intervalo de Endereços IP (179.248.0.0 - 179.248.255.255):**
>     *   Este é um bloco de endereços pertencente à Tim Celular S.A. A reputação de um bloco inteiro pode ser afetada se muitos IPs dentro dele forem consistentemente associados a atividades maliciosas. No entanto, a análise deve focar no IP específico.
> 
> 4.  **Verificadores de Reputação de IP:**
>     *   Ferramentas como AbuseIPDB, VirusTotal, e outras consultadas durante as buscas, podem fornecer um panorama mais amplo e atualizado da reputação do IP, agregando dados de diversas fontes, incluindo relatos de abuso, detecções por antivírus e participação em atividades como scanning, ataques de força bruta, ou distribuição de malware.
> 
> **Pontos Relevantes para Avaliação de Reputação:**
> 
> Com base nas buscas e na análise das informações fornecidas, os seguintes pontos são os mais relevantes para avaliar a reputação do IP 179.248.212.19:
> 
> *   **Presença em Blacklists (Indicador Chave):**
>     *   **DataPlane TELNET login blacklist:** A inclusão nesta lista é um forte indicador de atividade potencialmente maliciosa. Tentativas de login TELNET são frequentemente associadas a botnets ou atores mal-intencionados buscando comprometer dispositivos.
>     *   **Spamhaus PBL:** Sugere que o IP não deve ser usado para enviar e-mails diretamente, o que é comum para IPs dinâmicos. Embora não seja uma indicação direta de malícia, pode impactar a entregabilidade de e-mails.
> 
> *   **Natureza do IP (Dinâmico/Móvel):**
>     *   Sendo um IP da Tim Celular S.A., é provável que seja um IP dinâmico atribuído a um usuário de rede móvel. Isso significa que a atividade associada ao IP pode variar conforme o usuário ao qual ele está atribuído muda. No entanto, a listagem em blacklists como a DataPlane indica que, em algum momento recente, o IP foi usado para atividades suspeitas.
> 
> *   **Relatos de Abuso e Atividade Suspeita:**
>     *   Verificações em plataformas de reputação de IP podem revelar relatos específicos de abuso, como tentativas de hacking, scanning ou outras atividades maliciosas originadas deste IP.
> 
> **Conclusão Preliminar da Reputação:**
> 
> A presença do IP 179.248.212.19 na "DataPlane TELNET login blacklist" é o fator mais preocupante e sugere fortemente que este IP esteve envolvido em atividades maliciosas ou, no mínimo, altamente suspeitas, como tentativas de acesso não autorizado. A listagem na Spamhaus PBL é menos crítica para a malícia direta, mas corrobora a natureza de um IP que não deveria originar tráfego de e-mail direto.
> 
> Considerando que é um IP de uma operadora de telefonia móvel (Tim Celular S.A.), é possível que um dispositivo comprometido (como um smartphone infectado com malware) ou um usuário mal-intencionado tenha utilizado este IP para realizar as atividades que levaram à sua inclusão nas blacklists.
> 
> **Recomendação:**
> 
> Dada a listagem na DataPlane TELNET login blacklist, o IP 179.248.212.19 deve ser considerado de **reputação suspeita a potencialmente maliciosa**. Seria prudente monitorar ou bloquear o tráfego proveniente deste IP, especialmente em portas e protocolos sensíveis como o TELNET. É importante também verificar a data e hora exatas dos incidentes que levaram à listagem para entender se a atividade é recente e contínua.


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (avaliador) ---



> ## Análise de Reputação: IP 179.248.212.19
> 
> **Veredito:** O IP `179.248.212.19` é considerado **potencialmente malicioso**.
> 
> **Justificativa:**
> 
> A análise fornecida indica que o IP `179.248.212.19`, pertencente à Tim Celular S.A. no Brasil, está listado na **"DataPlane TELNET login blacklist"**. Esta inclusão é um forte indicador de atividade maliciosa, pois sinaliza que o IP foi observado tentando realizar logins via TELNET. O protocolo TELNET é conhecido por suas vulnerabilidades e é frequentemente explorado por atores mal-intencionados para tentativas de acesso não autorizado a dispositivos e sistemas.
> 
> Embora o IP possa ser dinâmico, comumente atribuído a usuários de redes móveis, a presença nesta blacklist específica sugere que, em algum momento recente, um dispositivo utilizando este IP esteve envolvido em atividades suspeitas ou maliciosas.
> 
> A listagem na **Spamhaus PBL (Policy Block List)**, embora não seja um indicador direto de malícia, reforça a natureza de um IP que não deveria originar tráfego de e-mail direto, o que é comum para IPs dinâmicos e pode ser explorado por máquinas comprometidas para envio de spam.
> 
> **Resumo Breve:**
> 
> O IP `179.248.212.19` foi classificado como **potencialmente malicioso**. A principal razão para esta classificação é sua presença na "DataPlane TELNET login blacklist", indicando envolvimento em tentativas de login não autorizado via TELNET, uma atividade característica de ataques. Recomenda-se cautela e monitoramento em relação a este IP.


--------------------------------------------------------------
